# Аналіз даних VHI
## Лабораторна робота №2

In [3]:
import os
import datetime
import urllib.request
import glob
import pandas as pd
from prettytable import PrettyTable

print("Середовище готове до роботи.")

Середовище готове до роботи.


### Функція завантаження даних з Інтернету
Функція завантажує CSV-файл з даними VHI за заданими параметрами та зберігає його у папці **data**.

In [4]:
def fetch_vhi_data(region_code, start_year=1981, end_year=2024):
    """Завантаження даних VHI для заданого коду регіону та збереження у вигляді CSV-файлу."""
    data_dir = "data"
    if not os.path.isdir(data_dir):
        os.makedirs(data_dir)
    else:
        # Якщо файл для даного регіону вже існує, завантаження не проводиться
        existing_files = [fname for fname in os.listdir(data_dir) if fname.startswith(f"vhi_id__{region_code}__")]
        if existing_files:
            print(f"Файл '{existing_files[0]}' вже присутній у папці '{data_dir}'. Завантаження пропущено.")
            return

    url = (f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={region_code}" 
           f"&year1={start_year}&year2={end_year}&type=Mean")
    response = urllib.request.urlopen(url)
    current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
    filename = f"vhi_id__{region_code}__{current_time}.csv"
    filepath = os.path.join(data_dir, filename)
    
    with open(filepath, 'wb') as f:
        f.write(response.read())
    
    print(f"Дані VHI збережено у файлі '{filepath}'.")

### Завантаження даних для тестування


In [5]:

for reg in range(1, 28):
     fetch_vhi_data(reg)

Дані VHI збережено у файлі 'data/vhi_id__1__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__2__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__3__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__4__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__5__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__6__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__7__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__8__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__9__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__10__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__11__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__12__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__13__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__14__2025-03-10_14-45.csv'.
Дані VHI збережено у файлі 'data/vhi_id__15__2025-03-10_1

### Відповідність регіональних кодів до назв областей
Словник для відображення коду регіону на його назву.

In [6]:
region_names = {
    1: 'Вінницька', 2: 'Волинська', 3: 'Дніпропетровська', 4: 'Донецька', 5: 'Житомирська',
    6: 'Закарпатська', 7: 'Запорізька', 8: 'Івано-Франківська', 9: 'Київська', 10: 'Кіровоградська',
    11: 'Луганська', 12: 'Львівська', 13: 'Миколаївська', 14: 'Одеська', 15: 'Полтавська',
    16: 'Рівненська', 17: 'Сумська', 18: 'Тернопільська', 19: 'Харківська', 20: 'Херсонська',
    21: 'Хмельницька', 22: 'Черкаська', 23: 'Чернівецька', 24: 'Чернігівська', 25: 'Республіка Крим'
}

### Формування єдиного DataFrame з даних
Функція зчитує всі CSV-файли з папки **data**, проводить попередню обробку даних та об'єднує їх у один DataFrame.

In [7]:
def build_dataframe(folder):
    """Зчитування та обробка CSV-файлів з папки, повертає об'єднаний DataFrame."""
    files = glob.glob(os.path.join(folder, "*.csv"))
    
    columns = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'unused']
    dataframes = []
    
    for file in files:
        try:
            # Отримання коду регіону з імені файлу
            reg_code = int(file.split('__')[1])
        except Exception as e:
            continue
        
        df = pd.read_csv(file, header=1, names=columns)
        df.at[0, 'Year'] = df.at[0, 'Year'][9:]
        
        df = df.drop(df.index[-1])
        df = df[df['VHI'] != -1]
        
        df = df.drop(columns=['unused'])
        
        df.insert(0, 'region_code', reg_code)
        df['Week'] = df['Week'].astype(int)
        
        dataframes.append(df)
    
    # Об'єднання всіх даних
    full_df = pd.concat(dataframes).drop_duplicates().reset_index(drop=True)
    
    full_df = full_df[(full_df['region_code'] != 12) & (full_df['region_code'] != 20)]
    
    remap = {
        1:22, 2:24, 3:23, 4:25, 5:3, 6:4, 7:8, 8:19, 9:20, 10:21,
        11:9, 13:10, 14:11, 15:12, 16:13, 17:14, 18:15, 19:16, 21:17,
        22:18, 23:6, 24:1, 25:2, 26:6, 27:5
    }
    full_df = full_df.replace({'region_code': remap})
    
    return full_df

df_main = build_dataframe('data')
print(df_main.head())

      region_code  Year  Week    SMN     SMT    VCI    TCI    VHI
2186           19  1982     1  0.038  252.22  47.05  69.15  58.10
2187           19  1982     2  0.036  253.39  45.39  63.52  54.45
2188           19  1982     3  0.033  254.22  41.22  60.22  50.72
2189           19  1982     4  0.031  255.60  37.35  58.01  47.68
2190           19  1982     5  0.030  257.16  32.54  56.92  44.73


### Аналіз показників VHI за регіонами
Функція визначає мінімальне та максимальне значення VHI для заданого регіону у вказаному діапазоні років.

In [8]:
def analyze_region_year(region_code, years=("1982", "2024")):
    """Аналіз VHI для конкретного регіону у заданому діапазоні років."""
    subset = df_main[(df_main["Year"].between(years[0], years[1])) & (df_main["region_code"] == region_code)]
    region_label = region_names.get(region_code, f"Регіон {region_code}")
    vhi_min = subset["VHI"].min()
    vhi_max = subset["VHI"].max()
    print(f"[+] {region_label}: min VHI = {vhi_min}, max VHI = {vhi_max}")

In [9]:
# Аналіз для всіх регіонів з 1 до 25
for code in range(1, 26):
    analyze_region_year(code)

[+] Вінницька: min VHI = 11.25, max VHI = 82.64
[+] Волинська: min VHI = 11.91, max VHI = 78.32
[+] Дніпропетровська: min VHI = 17.58, max VHI = 93.17
[+] Донецька: min VHI = 6.26, max VHI = 96.18
[+] Житомирська: min VHI = 19.48, max VHI = 77.57
[+] Закарпатська: min VHI = 10.88, max VHI = 96.69
[+] Запорізька: min VHI = nan, max VHI = nan
[+] Івано-Франківська: min VHI = 18.98, max VHI = 73.35
[+] Київська: min VHI = 10.6, max VHI = 80.88
[+] Кіровоградська: min VHI = 16.36, max VHI = 84.52
[+] Луганська: min VHI = 12.45, max VHI = 90.32
[+] Львівська: min VHI = 18.31, max VHI = 69.96
[+] Миколаївська: min VHI = 5.94, max VHI = 92.31
[+] Одеська: min VHI = 5.52, max VHI = 89.14
[+] Полтавська: min VHI = 15.68, max VHI = 85.14
[+] Рівненська: min VHI = 20.7, max VHI = 77.45
[+] Сумська: min VHI = 12.64, max VHI = 81.96
[+] Тернопільська: min VHI = 19.45, max VHI = 77.71
[+] Харківська: min VHI = 9.36, max VHI = 91.42
[+] Херсонська: min VHI = 12.23, max VHI = 90.61
[+] Хмельницька: mi

In [10]:
print(df_main['region_code'].unique())


[19 10 17 24 21  3  1 25 23 11 13  2  6  8  5 15 18  9 16 22  4 20 14 12]


### Визначення років з посухою
Функція обчислює роки, в яких спостерігалася посуха (екстремальна або помірна) на основі заданого порогового значення.

In [11]:
def identify_drought_years(data, perc_threshold, drought_category, start_year=None, end_year=None):
    """Визначення років з посухою за заданим порогом у відсотках.
    drought_category: 'extreme' для VHI < 15, 'moderate' для VHI від 15 до 35.
    """
    if start_year is not None:
        data = data[data["Year"] >= start_year]
    if end_year is not None:
        data = data[data["Year"] <= end_year]
    
    summary = {}
    for yr, group in data.groupby("Year"):
        total = group["region_code"].nunique()
        if drought_category == "extreme":
            affected = group[group["VHI"] < 15]["region_code"].nunique()
        elif drought_category == "moderate":
            affected = group[(group["VHI"] > 15) & (group["VHI"] < 35)]["region_code"].nunique()
        else:
            continue
        
        if (affected / total) > (perc_threshold / 100):
            summary[yr] = (affected, round((affected / total) * 100, 2))
    
    table = PrettyTable()
    if drought_category == "extreme":
        table.title = f"Роки з екстремальною посухою (більше {perc_threshold}% регіонів мають VHI < 15)"
    elif drought_category == "moderate":
        table.title = f"Роки з помірною посухою (більше {perc_threshold}% регіонів мають VHI від 15 до 35)"
    table.field_names = ["Year", "К-сть регіонів", "Відсоток"]
    for yr, (cnt, pct) in summary.items():
        table.add_row([yr, cnt, f"{pct}%"])
    
    return table

In [12]:
# Отримання таблиць для екстремальних та помірних умов посухи
extreme_drought = identify_drought_years(df_main, 10, "extreme", start_year="2000", end_year="2024")
print(extreme_drought)

moderate_drought = identify_drought_years(df_main, 10, "moderate", start_year="2000", end_year="2024")
print(moderate_drought)

+-------------------------------------------------------------------+
| Роки з екстремальною посухою (більше 10% регіонів мають VHI < 15) |
+-----------+---------------------------------+---------------------+
|    Year   |          К-сть регіонів         |       Відсоток      |
+-----------+---------------------------------+---------------------+
|    2000   |                4                |        16.67%       |
|    2007   |                5                |        20.83%       |
+-----------+---------------------------------+---------------------+
+----------------------------------------------------------------------+
| Роки з помірною посухою (більше 10% регіонів мають VHI від 15 до 35) |
+-----------+-----------------------------------+----------------------+
|    Year   |           К-сть регіонів          |       Відсоток       |
+-----------+-----------------------------------+----------------------+
|    2000   |                 24                |        100.0%        |
| 